# ECG arrhythmia classification using CNN

In [1]:
pwd

'C:\\Users\\Muthamizhan\\Desktop\\Application Building'

In [2]:
!pip install keras
!pip install tensorflow

In [3]:
import keras
keras.__version__

'2.9.0'

In [4]:
import tensorflow
tensorflow.__version__

'2.9.1'

### Importing Neccessary Libraries

In [5]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator


### Image Data Agumentation

In [6]:
#setting parameter for Image Data agumentation to the traing data
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [7]:
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [8]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_69c05974e5c84795a978662af2736fc1 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_69c05974e5c84795a978662af2736fc1 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_69c05974e5c84795a978662af2736fc1 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='KNnyzgYsEAWBFEGgzWg_6j5CyAcjpaOlCPz3gd9KEdr_',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_69c05974e5c84795a978662af2736fc1)

streaming_body_1 = client_69c05974e5c84795a978662af2736fc1.get_object(Bucket='ecgimagebasedheartbeatclassificat-donotdelete-pr-l2ugdciyflqayf', Key='data.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


EndpointConnectionError: Could not connect to the endpoint URL: "https://s3.private.us.cloud-object-storage.appdomain.cloud/ecgimagebasedheartbeatclassificat-donotdelete-pr-l2ugdciyflqayf/data.zip"

In [ ]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
print(file_paths)
for path in file_paths:
    unzip.extract(path)

In [ ]:
pwd

In [14]:
import os
filenames = os.listdir('/home/wsuser/work/data')

In [15]:
#performing data agumentation to train data
x_train=train_datagen.flow_from_directory('/home/wsuser/work/data/train',target_size=(64,64),batch_size=32,class_mode='categorical')
#performing data agumentation to test data
x_test=test_datagen.flow_from_directory('/home/wsuser/work/data/test',target_size=(64,64),batch_size=32,class_mode='categorical')

Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.


In [16]:
print(x_train.class_indices)#checking the number of classes

{'Left Bundle Branch Block': 0, 'Normal': 1, 'Premature Atrial Contraction': 2, 'Premature Ventricular Contractions': 3, 'Right Bundle Branch Block': 4, 'Ventricular Fibrillation': 5}


In [17]:
from collections import Counter as c
c(x_train.labels)

Counter({0: 504, 1: 7346, 2: 2054, 3: 2759, 4: 2239, 5: 439})

### Creating the model

In [18]:
# create model
model=Sequential()
# adding model layer
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))#convolutional layer
model.add(MaxPooling2D(pool_size=(2,2))) #MaxPooling2D-for downsampling the input

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())#flatten the dimension of the image
model.add(Dense(32))#deeply connected neural network layers.
model.add(Dense(6,activation='softmax'))#output layer with 6 neurons


2022-06-03 15:08:02.006929: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow
2022-06-03 15:08:02.006997: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


In [19]:
model.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 32)                2

### Compiling the model

In [20]:
# Compile model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

### Fitting the model

In [21]:
# Fit the model 
model.fit_generator(generator=x_train,steps_per_epoch = len(x_train),
                    epochs=10, validation_data=x_test,validation_steps = len(x_test))

/tmp/wsuser/ipykernel_2832/1916141677.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=x_train,steps_per_epoch = len(x_train),


Epoch 1/10
480/480 [==============================] - 97s 201ms/step - loss: 0.7450 - accuracy: 0.7491 - val_loss: 0.4413 - val_accuracy: 0.8504
Epoch 2/10
480/480 [==============================] - 96s 201ms/step - loss: 0.2724 - accuracy: 0.9186 - val_loss: 0.4429 - val_accuracy: 0.8809
Epoch 3/10
480/480 [==============================] - 97s 201ms/step - loss: 0.2204 - accuracy: 0.9347 - val_loss: 0.2756 - val_accuracy: 0.9207
Epoch 4/10
480/480 [==============================] - 97s 201ms/step - loss: 0.1896 - accuracy: 0.9435 - val_loss: 0.2367 - val_accuracy: 0.9314
Epoch 5/10
480/480 [==============================] - 96s 201ms/step - loss: 0.1637 - accuracy: 0.9497 - val_loss: 0.2607 - val_accuracy: 0.9174
Epoch 6/10
480/480 [==============================] - 96s 200ms/step - loss: 0.1526 - accuracy: 0.9544 - val_loss: 0.2423 - val_accuracy: 0.9263
Epoch 7/10
480/480 [==============================] - 96s 199ms/step - loss: 0.1366 - accuracy: 0.9567 - val_loss: 0.2960 - val_ac

In [22]:
# model.fit_generator(x_train,epochs=10,validation_data=x_test)

### Saving our model

In [23]:
# Save the model
from tensorflow.keras.models import load_model
model.save('ECG.h5')

In [24]:
!tar -zcvf ECG-Image-based-heartbeat-classification-model_new.tgz ECG.h5

ECG.h5


In [25]:
ls -1

data/
ECG.h5
ECG-Image-based-heartbeat-classification-model_new.tgz


In [26]:
!pip install watson-machine-learning-client --upgrade

In [27]:
# Replace the credentials that you got from watson machine learning service
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                    "apikey":"WpWfHtY_VXAXAgD4uzxBAb00C2FJstDEVyb3oXk1UaRm"
                  }
client = APIClient(wml_credentials)


In [28]:
client = APIClient(wml_credentials)

In [29]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [30]:
space_uid = guid_from_space_name(client, 'image_classification')
print("Space UID = "+ space_uid)

Space UID = 26b6a24d-f745-4d09-b456-8f6dfd7d9ca6


In [31]:
client.set.default_space(space_uid)

'SUCCESS'

In [32]:
client.software_specifications.list(limit=100)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1

In [33]:
import tensorflow
tensorflow.__version__

'2.9.1'

In [34]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_2.4-py3.7")
software_spec_uid

'65e171d7-72d1-55d9-8ebb-f813d620c9bb'

In [38]:
model_details = client.repository.store_model(model='ECG-Image-based-heartbeat-classification-model_new.tgz',meta_props={
client.repository.ModelMetaNames.NAME:"image_classification",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_uid_by_name("tensorflow_2.4-py3.7"),
client.repository.ModelMetaNames.TYPE:"keras_2.2.5"})

model_id = client.repository.get_model_uid(model_details)
    

In [41]:
client.repository.download(model_id, 'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


In [42]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [43]:
model = load_model("ECG.h5")